In [142]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
from get_data import get_data, read_data
from jia_cidades import F_Quarentena
from jia import F, tot_deaths
from jia import new_deaths as new_deaths_1_cidade
from jia import new_diagn as new_diagn_1_cidade
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [60]:
df_conf = pd.read_csv('data_municipios/BIConfirmadosCOVID_RJ16abr.csv',encoding = 'ISO-8859-1', delimiter = ';' )

In [61]:
df_conf = read_data(df_conf, 'confirmados')
municipios = 'all'
skip = False
dt_start = '29-03-2020'
dt_fim = '12-04-2020'
header = ['Data', 'Municipio', 'Casos']

In [62]:
    if municipios == 'all':
        municipios = set(df_conf['Municipio'])
    else:
        if skip == True:
            municipios = set(df_conf[df_conf['Municipio'].str.contains('|'.join(municipios))])
        else:
            municipios = set(df_conf[~df_conf['Municipio'].str.contains('|'.join(municipios))])

    ## Configurando os dias
    dt_start = dt.datetime.strptime(dt_start, '%d-%m-%Y')
    dt_fim = dt.datetime.strptime(dt_fim, '%d-%m-%Y')
    days = mdates.drange(dt_start, dt_fim, dt.timedelta(days = 1))
    
    dt_start = dt.datetime.strftime(dt_start, '%d-%m-%Y')
    dt_fim = dt.datetime.strftime(dt_fim, '%d-%m-%Y')
    dates = [dt.datetime.strftime(mdates.num2date(i), '%d-%m-%Y').replace('-','/') for i in days]

    ## Extraindo os dados e organizando em listas para o Dicionario
    casos = []
    nome_m = []
    data_m = []
    for m in municipios:
        lst = []
        lst_m = [m]*len(dates)
        lst_d = []
        for d in dates:
            df = df_conf[df_conf['Municipio'] == m]
            lst.append(len(df[df['Data'] == d]))
            lst_d.append(d)
        nome_m.append(lst_m)
        casos.append(lst)
        data_m.append(lst_d)

    ## Criando o Dicionario
    lst = [[],[],[]]
    for c, m, d in zip(casos, nome_m, data_m):
        lst[0] += d
        lst[1] += m
        lst[2] += c

    dic = {}
    for i,h in enumerate(header):
        dic.update({h: lst[i]})

    ## Transformando o dicionario em DataFrame
    df = pd.DataFrame(dic)

In [63]:
fig = px.line(df, x = 'Data', y = 'Casos', line_group = 'Municipio', render_mode = 'svg', color = 'Municipio')
fig.update_layout(title = 'Casos confirmados por Municipio do Estado do Rio de Janeiro')
fig.show()

In [64]:
lst5 = ['RIO DE JANEIRO', 'NITEROI']

In [65]:
for m in lst5:
    df_conf = df_conf[df_conf['Municipio'] != m | ]

SyntaxError: invalid syntax (<ipython-input-65-e44c4ad9148e>, line 2)

In [ ]:
'|'.join(lst5)

In [ ]:
df2 = df_conf[~df_conf['Municipio'].str.contains('|'.join(lst5))]

In [ ]:
df2[df2['Municipio'] == 'DUQUE DE CAXIAS']

In [ ]:
municipios = set(df_conf['Municipio'])

In [ ]:
header = ['Data', 'Municipio', 'Casos']

In [ ]:
date_start = '29-03-2020'
date_fim = '12-04-2020'

date_start = dt.datetime.strptime(date_start, '%d-%m-%Y')
date_fim = dt.datetime.strptime(date_fim, '%d-%m-%Y')
days = mdates.drange(date_start, date_fim, dt.timedelta(days = 1))

date_start = dt.datetime.strftime(date_start, '%d-%m-%Y')
date_fim = dt.datetime.strftime(date_fim, '%d-%m-%Y')

In [ ]:
dates = [dt.datetime.strftime(mdates.num2date(i), '%d-%m-%Y').replace('-','/') for i in days]

In [ ]:
casos = []
nome_m = []
data_m = []
for m in municipios:
    lst = []
    lst_m = [m]*len(dates)
    lst_d = []
    for d in dates:
        df = df_conf[df_conf['Municipio'] == m]
        lst.append(len(df[df['Data'] == d]))
        lst_d.append(d)
    nome_m.append(lst_m)
    casos.append(lst)
    data_m.append(lst_d)

In [ ]:
lst = [[],[],[]]
for c, m, d in zip(casos, nome_m, data_m):
    lst[0] += d
    lst[1] += m
    lst[2] += c

In [ ]:
dic = {}

In [ ]:
for i,h in enumerate(header):
        dic.update({h: lst[i]})

In [ ]:
df = pd.DataFrame(dic)

In [ ]:
df2 = df[df['Municipio'] != 'RIO DE JANEIRO']

In [ ]:
fig = px.line(df2, x = 'Data', y = 'Casos', line_group = 'Municipio', render_mode = 'svg', color = 'Municipio')
fig.update_layout(title = 'Casos confirmados por Municipio do Estado do Rio de Janeiro')
fig.show()

In [66]:
# Taxa de subnotificação
taxa_subn   = 8
# Proporção de infectados assintomáticos
prop_assint = 0.80

In [67]:
# Idx       :         0,    1,     2,    3,     4,    5,     6,      7,    8,      9,     10,     11,     12,    13
# Parâmetros:      betaS, theta,  p, lambda, sigma, rho,  epsA, gammaA, epsI, gammaI, deathI, gammaD, deathD, betaQ
params_rio_base = (1e-7,  1/10, 1/4,  1/6,   1/7, 0.94, 1/100, 0.0996, 1/30, 0.0766, 0.0026, 0.0843, 0.0017, 1e-7/7)

In [68]:
# Dados para a cidade do Rio de Janeiro,
# IBGE 2019 (estimativa)
pop_Rio = 6718903
# 29 de Março
diagn_Rio, recup_Rio = 489, 16
infect_Rio = diagn_Rio*(taxa_subn-1)*(1 - prop_assint)
assint_Rio = diagn_Rio*(taxa_subn-1)*prop_assint
expostos_Rio = diagn_Rio*7
quarentena_Rio = 0.6 * pop_Rio

suscetiveis_Rio = pop_Rio - (quarentena_Rio + expostos_Rio + assint_Rio + infect_Rio + diagn_Rio + recup_Rio)

CI = [suscetiveis_Rio, quarentena_Rio, expostos_Rio, assint_Rio, infect_Rio, diagn_Rio, recup_Rio]

In [69]:
# Do dia 29/04 até o dia 12/04, dados da prefeitura
dados_Rio = [
    [26, 38, 30, 114, 110, 60, 115, 86, 42, 141, 198, 184, 175, 97, 91, 326, 71, 126],
    [1, 3, 2, 4,   7,  5,   2,  4,  5,  12,  14,   9,  10,  6,  8,   9, 25,  27]
]

In [70]:
T_fim = len(dados_Rio[0])

start = dt.datetime.strptime("29-03-2020", "%d-%m-%Y")
then = start + dt.timedelta(days=T_fim)
days = mdates.drange(start,then,dt.timedelta(days=1))

In [71]:
# Supondo mantida a quarentena
f_rio_base = lambda t,v: F_Quarentena(t,v, *params_rio_base)
results_base_quarentena = solve_ivp(f_rio_base, (1,T_fim), CI, t_eval=range(1,T_fim+1))
g_rio_base = lambda t, v: F(t, v, *params_rio_base[:-1])
results_base_jia = solve_ivp(g_rio_base, (1, T_fim), CI, t_eval = range(1, T_fim+1))

In [72]:
diagn_quarentena = new_diagn_1_cidade(results_base_quarentena.y, params_rio_base)
deaths_quarentena = new_deaths_1_cidade(results_base_quarentena.y, params_rio_base)

diagn_jia = new_diagn_1_cidade(results_base_jia.y, params_rio_base)
deaths_jia = new_deaths_1_cidade(results_base_jia.y, params_rio_base)

titles = ['Novos diagnósticos: Quarentena tipo Rio', 'Novos diagnósticos: Quarentena tipo Jia', 
          'Diferença de diagnósticos entre tipos de quarentena', 'Novos óbitos: Quarentena tipo Rio', 
          'Novos óbitos: Quarentena tipo Jia', 'Diferença de óbitos entre tipos de quarentena']

In [77]:
results_base_quarentena.y

array([[2.68021020e+06, 2.68144066e+06, 2.68214287e+06, 2.68245307e+06,
        2.68250634e+06, 2.68242041e+06, 2.68220586e+06, 2.68188462e+06,
        2.68149150e+06, 2.68104640e+06, 2.68053858e+06, 2.67996406e+06,
        2.67932658e+06, 2.67863735e+06, 2.67789188e+06, 2.67707856e+06,
        2.67619267e+06, 2.67522974e+06],
       [4.03134180e+06, 4.02974456e+06, 4.02857673e+06, 4.02771121e+06,
        4.02701990e+06, 4.02638966e+06, 4.02580858e+06, 4.02525256e+06,
        4.02468421e+06, 4.02407956e+06, 4.02344259e+06, 4.02277007e+06,
        4.02205069e+06, 4.02126543e+06, 4.02040910e+06, 4.01948231e+06,
        4.01847825e+06, 4.01738889e+06],
       [3.42300000e+03, 3.31021155e+03, 3.30388577e+03, 3.38267416e+03,
        3.52843169e+03, 3.72674139e+03, 3.97274347e+03, 4.26256274e+03,
        4.59223734e+03, 4.95978993e+03, 5.36707610e+03, 5.81595563e+03,
        6.30784702e+03, 6.84376282e+03, 7.42678754e+03, 8.06122599e+03,
        8.75101474e+03, 9.50039626e+03],
       [2.738

In [101]:
sol = results_base_quarentena.y
keys = ['Suscetiveis', 'Quarentenados', 'Expostos', 'Assintomáticos', 'Infectados', 'Diagnosticados', 'Recuperados']
comp = [len(sol[0])*[key] for key in keys]
value_comp = []
valores_simul = []
for c, y in zip(comp, sol):
    value_comp += c
    valores_simul += list(y)

In [112]:
dt_start = '29-03-2020'
dt_fim = '16-04-2020'
dt_start = dt.datetime.strptime(dt_start, '%d-%m-%Y')
dt_fim = dt.datetime.strptime(dt_fim, '%d-%m-%Y')
days = mdates.drange(dt_start, dt_fim, dt.timedelta(days = 1))

dt_start = dt.datetime.strftime(dt_start, '%d-%m-%Y')
dt_fim = dt.datetime.strftime(dt_fim, '%d-%m-%Y')
dates = [dt.datetime.strftime(mdates.num2date(i), '%d-%m-%Y').replace('-','/') for i in days]
dates*= len(sol)
#dates

In [113]:
len(dates), len(value_comp), len(valores_simul)

(126, 126, 126)

In [114]:
dic = {}
header = ['Data', 'Valores', 'Compartimentos']
values = [dates, valores_simul, value_comp]
for i, h in enumerate(header):
    dic.update({h:values[i]})

In [115]:
df = pd.DataFrame(dic)
df

,Data,Valores,Compartimentos
0,29/03/2020,2.680210e+06,Suscetiveis
1,30/03/2020,2.681441e+06,Suscetiveis
2,31/03/2020,2.682143e+06,Suscetiveis
3,01/04/2020,2.682453e+06,Suscetiveis
4,02/04/2020,2.682506e+06,Suscetiveis
...,...,...,...
121,11/04/2020,5.439193e+03,Recuperados
122,12/04/2020,5.992420e+03,Recuperados
123,13/04/2020,6.581875e+03,Recuperados
124,14/04/2020,7.211738e+03,Recuperados


In [125]:
fig = go.Figure()
fig.add_shape(px.line(df, line_group='Compartimentos', x = 'Data', y = 'Valores', color = 'Compartimentos'))
fig.update_layout(yaxis_type="log")
fig.show()

ValueError: The first argument to the plotly.graph_objs.layout.Shape 
constructor must be a dict or 
an instance of :class:`plotly.graph_objs.layout.Shape`

In [88]:
dic = {}

for i, sol in enumerate(results_base_quarentena.y):
    dic.update({keys[i]:sol})

In [85]:
df = pd.DataFrame(dic)

In [86]:
df

,Suscetiveis,Quarentenados,Expostos,Assintomáticos,Infectados,Diagnosticados,Recuperados
0,2.680210e+06,4.031342e+06,3423.000000,2738.400000,684.600000,489.000000,16.000000
1,2.681441e+06,4.029745e+06,3310.211553,2481.380339,1038.002512,501.392944,383.706688
2,2.682143e+06,4.028577e+06,3303.885767,2250.624364,1347.373647,520.858403,753.600823
3,2.682453e+06,4.027711e+06,3382.674162,2044.077010,1627.629759,546.147252,1126.356110
4,2.682506e+06,4.027020e+06,3528.431695,1859.860015,1891.918877,576.231724,1502.931277
5,2.682420e+06,4.026390e+06,3726.741393,1696.238123,2151.314583,610.350469,1884.651539
6,2.682206e+06,4.025809e+06,3972.743466,1551.410395,2411.355531,648.471479,2273.939061
7,2.681885e+06,4.025253e+06,4262.562744,1423.766576,2677.188536,690.619843,2673.297166
8,2.681492e+06,4.024684e+06,4592.237341,1311.894172,2954.217371,736.800174,3085.211098
9,2.681046e+06,4.024080e+06,4959.789930,1214.497147,3246.893037,787.162723,3512.421628


In [140]:
x0 = np.linspace(1, 3, 200)
y0 = x0 * np.sin(np.power(x0, 2)) + 1
colors = ['blue', 'red', 'green', 'orange', 'grey']

In [213]:
fig = make_subplots(rows=2, cols = 2, column_width = [1,1], row_width=[1,1], shared_xaxes= True)
for i in range(2,7):
    fig.add_trace(go.Scatter(
    x=np.arange(len(sol[i])),
    y=sol[i], name = keys[i]), row = 1, col = 1)
    
for i in range(2,7):
    fig.add_trace(go.Scatter(
    x=np.arange(len(sol[i])),
    y=sol[i], name = keys[i]), row = 2, col = 1)
    
fig.update_layout(
    shapes=[
        # 1st highlight during Feb 4 - Feb 6
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=3,
            y0=0.575,
            x1=5,
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
    ]
)
fig.update_layout(height=800, width=1000)
fig.show()

In [215]:
dados1 = np.array([[2.67708320e+06, 2.67884253e+06, 2.67986161e+06, 2.68034403e+06,
        2.68048189e+06, 2.68042442e+06, 5.22425951e+06, 6.16078660e+06,
        4.97355483e+06, 4.18954658e+06, 3.67469879e+06, 3.33362390e+06,
        3.10873871e+06, 2.96081571e+06, 2.86145905e+06, 2.79666359e+06,
        2.75365674e+06, 2.72366307e+06, 2.70391627e+06, 2.69105684e+06,
        2.68122603e+06, 2.67369018e+06, 2.66844246e+06, 2.66479661e+06,
        2.66145595e+06, 2.65782638e+06, 2.65447101e+06, 2.65156389e+06,
        2.64902118e+06, 2.64650120e+06, 2.64343641e+06],
       [4.03134180e+06, 4.02915229e+06, 4.02758993e+06, 4.02646545e+06,
        4.02559812e+06, 4.02484656e+06, 1.48192726e+06, 5.45725782e+05,
        1.73069354e+06, 2.51258139e+06, 3.02537418e+06, 3.36438398e+06,
        3.58715612e+06, 3.73288522e+06, 3.82993303e+06, 3.89229294e+06,
        3.93271564e+06, 3.95995665e+06, 3.97677609e+06, 3.98652033e+06,
        3.99302854e+06, 3.99702396e+06, 3.99850433e+06, 3.99814169e+06,
        3.99721358e+06, 3.99629634e+06, 3.99481456e+06, 3.99257879e+06,
        3.98965433e+06, 3.98636120e+06, 3.98324237e+06],
       [4.87900000e+03, 4.63097202e+03, 4.52064590e+03, 4.51769975e+03,
        4.59688642e+03, 4.73902851e+03, 5.34200704e+03, 6.49829294e+03,
        7.73798589e+03, 8.68163308e+03, 9.43754172e+03, 1.01044740e+04,
        1.07275132e+04, 1.13424407e+04, 1.19848739e+04, 1.26584569e+04,
        1.33807198e+04, 1.41658496e+04, 1.50084699e+04, 1.59127234e+04,
        1.68913623e+04, 1.79436700e+04, 1.90666529e+04, 2.02637398e+04,
        2.15444593e+04, 2.29154271e+04, 2.43748993e+04, 2.59244732e+04,
        2.75689392e+04, 2.93162806e+04, 3.11773453e+04],
       [3.90320000e+03, 3.53653741e+03, 3.20656508e+03, 2.91035884e+03,
        2.64520288e+03, 2.40853593e+03, 2.25838179e+03, 2.11679859e+03,
        1.95523275e+03, 1.81909158e+03, 1.70407725e+03, 1.60661809e+03,
        1.52450706e+03, 1.45599579e+03, 1.39954434e+03, 1.35441214e+03,
        1.31965545e+03, 1.29448950e+03, 1.27861624e+03, 1.27158870e+03,
        1.27283525e+03, 1.28213829e+03, 1.29937482e+03, 1.32433939e+03,
        1.35675225e+03, 1.39645027e+03, 1.44350037e+03, 1.49794496e+03,
        1.55980381e+03, 1.62907411e+03, 1.70573774e+03],
       [9.75800000e+02, 1.47397189e+03, 1.89769404e+03, 2.26839811e+03,
        2.60424573e+03, 2.91948143e+03, 3.24095543e+03, 3.64583399e+03,
        4.16736138e+03, 4.76867908e+03, 5.41535360e+03, 6.08100491e+03,
        6.75758626e+03, 7.44151906e+03, 8.13003626e+03, 8.83060593e+03,
        9.54559688e+03, 1.02780846e+04, 1.10370354e+04, 1.18277475e+04,
        1.26523476e+04, 1.35172479e+04, 1.44291909e+04, 1.53928555e+04,
        1.64110069e+04, 1.74875284e+04, 1.86289173e+04, 1.98407235e+04,
        2.11277639e+04, 2.24941217e+04, 2.39432545e+04],
       [6.97000000e+02, 7.14612124e+02, 7.41965200e+02, 7.76856630e+02,
        8.17504296e+02, 8.62642472e+02, 9.12238671e+02, 9.67790014e+02,
        1.03212550e+03, 1.10791677e+03, 1.19609219e+03, 1.29711041e+03,
        1.41037683e+03, 1.53525194e+03, 1.67129976e+03, 1.81771078e+03,
        1.97424711e+03, 2.14082950e+03, 2.31724466e+03, 2.50366254e+03,
        2.70056817e+03, 2.90832372e+03, 3.12742365e+03, 3.35855442e+03,
        3.60257970e+03, 3.86037653e+03, 4.13278822e+03, 4.42078932e+03,
        4.72546019e+03, 5.04798696e+03, 5.38965596e+03],
       [2.30000000e+01, 5.46973565e+02, 1.07315654e+03, 1.60136925e+03,
        2.13193173e+03, 2.66581762e+03, 3.19353675e+03, 3.74279566e+03,
        4.32812170e+03, 4.94807858e+03, 5.61006305e+03, 6.32007961e+03,
        7.08160596e+03, 7.89765984e+03, 8.77137855e+03, 9.70488115e+03,
        1.07013247e+04, 1.17641115e+04, 1.28962800e+04, 1.41017228e+04,
        1.53850916e+04, 1.67508537e+04, 1.82038545e+04, 1.97495772e+04,
        2.13941201e+04, 2.31438461e+04, 2.50052142e+04, 2.69852629e+04,
        2.90916032e+04, 3.13324192e+04, 3.37164547e+04],
       [1.93950800e+05, 1.94132725e+05, 1.94237382e+05, 1.94285950e+05,
        1.94298433e+05, 1.94290366e+05, 3.76251858e+05, 4.41479323e+05,
        3.56387794e+05, 3.00181883e+05, 2.63258084e+05, 2.38781191e+05,
        2.22625721e+05, 2.11980870e+05, 2.04811206e+05, 2.00114027e+05,
        1.96973621e+05, 1.94760682e+05, 1.93277638e+05, 1.92283618e+05,
        1.91501307e+05, 1.90878233e+05, 1.90414108e+05, 1.90059493e+05,
        1.89720763e+05, 1.89354825e+05, 1.89002216e+05, 1.88675361e+05,
        1.88367924e+05, 1.88054800e+05, 1.87694453e+05],
       [2.92549200e+05, 2.92323148e+05, 2.92161782e+05, 2.92045458e+05,
        2.91955389e+05, 2.91876876e+05, 1.07467525e+05, 3.95753562e+04,
        1.24416093e+05, 1.80379081e+05, 2.17062559e+05, 2.41295068e+05,
        2.57198571e+05, 2.67580534e+05, 2.74472695e+05, 2.78876542e+05,
        2.81705319e+05, 2.83586064e+05, 2.84715688e+05, 2.85333562e+05,
        2.85714729e+05, 2.85910480e+05, 2.85920090e+05, 2.85791374e+05,
        2.85615751e+05, 2.85434284e+05, 2.85205082e+05, 2.84914010e+05,
        2.84565339e+05, 2.84181749e+05, 2.83802024e+05],
       [5.04000000e+02, 4.78864711e+02, 4.68710356e+02, 4.70264522e+02,
        4.80801599e+02, 4.98251446e+02, 6.37514420e+02, 8.71927630e+02,
        9.92118570e+02, 1.08875322e+03, 1.16961879e+03, 1.24354865e+03,
        1.31459603e+03, 1.38621905e+03, 1.46215303e+03, 1.54254643e+03,
        1.62926754e+03, 1.72385741e+03, 1.82556083e+03, 1.93477923e+03,
        2.05296346e+03, 2.17996000e+03, 2.31534523e+03, 2.45946835e+03,
        2.61341285e+03, 2.77791241e+03, 2.95268185e+03, 3.13782959e+03,
        3.33384418e+03, 3.54159420e+03, 3.76228805e+03],
       [4.03200000e+02, 3.99352090e+02, 3.95431020e+02, 3.91541824e+02,
        3.87774702e+02, 3.84201875e+02, 3.19043971e+02, 2.72015254e+02,
        2.74535106e+02, 2.77904379e+02, 2.81967372e+02, 2.86587973e+02,
        2.91728396e+02, 2.97374438e+02, 3.03515314e+02, 3.10206727e+02,
        3.17474110e+02, 3.25349540e+02, 3.33906236e+02, 3.43196445e+02,
        3.53254183e+02, 3.64143769e+02, 3.75934689e+02, 3.88684989e+02,
        4.02442169e+02, 4.17262991e+02, 4.33224228e+02, 4.50398680e+02,
        4.68856198e+02, 4.88663676e+02, 5.09885786e+02],
       [1.00800000e+02, 1.52283964e+02, 1.96191362e+02, 2.34800460e+02,
        2.70041601e+02, 3.03426259e+02, 3.42005988e+02, 4.00595658e+02,
        4.76789199e+02, 5.58344139e+02, 6.42688641e+02, 7.27617224e+02,
        8.12708488e+02, 8.97899528e+02, 9.83107633e+02, 1.06938579e+03,
        1.15713727e+03, 1.24681201e+03, 1.33953419e+03, 1.43597438e+03,
        1.53641052e+03, 1.64161933e+03, 1.75240714e+03, 1.86932683e+03,
        1.99269584e+03, 2.12295594e+03, 2.26086061e+03, 2.40704438e+03,
        2.56204403e+03, 2.72629859e+03, 2.90016255e+03],
       [7.20000000e+01, 7.39880706e+01, 7.72771323e+01, 8.15867344e+01,
        8.66876437e+01, 9.24120045e+01, 9.84230639e+01, 1.04935493e+02,
        1.12915454e+02, 1.22807043e+02, 1.34555586e+02, 1.48097117e+02,
        1.63282851e+02, 1.79980055e+02, 1.98094380e+02, 2.17501569e+02,
        2.38151246e+02, 2.60018115e+02, 2.83067471e+02, 3.07312386e+02,
        3.32805157e+02, 3.59586546e+02, 3.87714842e+02, 4.17271294e+02,
        4.48358321e+02, 4.81080705e+02, 5.15539563e+02, 5.51851014e+02,
        5.90143326e+02, 6.30556914e+02, 6.73243641e+02],
       [2.00000000e+00, 2.18917385e+01, 4.55924045e+01, 7.27535442e+01,
        1.03108662e+02, 1.36487653e+02, 1.87744464e+02, 2.38297366e+02,
        2.83753871e+02, 3.36077530e+02, 3.95678108e+02, 4.62900802e+02,
        5.37886785e+02, 6.20770096e+02, 7.11729757e+02, 8.10863163e+02,
        9.18400276e+02, 1.03461573e+03, 1.15976130e+03, 1.29419357e+03,
        1.43836248e+03, 1.59270823e+03, 1.75772140e+03, 1.93396800e+03,
        2.12208680e+03, 2.32275075e+03, 2.53664864e+03, 2.76453533e+03,
        3.00723002e+03, 3.26561634e+03, 3.54064085e+03]])

dados2 = np.array([[2.67708320e+06, 2.67884253e+06, 2.67986161e+06, 2.68034403e+06,
        2.68048189e+06, 2.68043405e+06, 2.68022075e+06, 2.67987541e+06,
        2.67945197e+06, 2.67898036e+06, 2.67844666e+06, 2.67784845e+06,
        2.67719436e+06, 2.67650344e+06, 2.67577585e+06, 2.67499653e+06,
        2.67415782e+06, 2.67325618e+06, 2.67229209e+06, 2.67127017e+06,
        2.67019718e+06, 2.66906471e+06, 2.66785704e+06, 2.66656190e+06,
        2.66517053e+06, 2.66367771e+06, 2.66208174e+06, 2.66038445e+06,
        2.65859120e+06, 2.65670942e+06, 2.65471281e+06],
       [4.03134180e+06, 4.02915229e+06, 4.02758993e+06, 4.02646545e+06,
        4.02559812e+06, 4.02483734e+06, 4.02416509e+06, 4.02354905e+06,
        4.02293618e+06, 4.02229579e+06, 4.02163823e+06, 4.02096217e+06,
        4.02025529e+06, 4.01949494e+06, 4.01867596e+06, 4.01780699e+06,
        4.01688914e+06, 4.01591927e+06, 4.01488994e+06, 4.01378946e+06,
        4.01260345e+06, 4.01133118e+06, 4.00997889e+06, 4.00854892e+06,
        4.00703951e+06, 4.00544478e+06, 4.00375477e+06, 4.00195541e+06,
        4.00002854e+06, 3.99795331e+06, 3.99574081e+06],
       [4.87900000e+03, 4.63097202e+03, 4.52064590e+03, 4.51769975e+03,
        4.59688642e+03, 4.73828658e+03, 4.93265794e+03, 5.17241189e+03,
        5.44998444e+03, 5.76041959e+03, 6.10346596e+03, 6.47876893e+03,
        6.88535583e+03, 7.32168357e+03, 7.78814208e+03, 8.28725134e+03,
        8.82112406e+03, 9.39155907e+03, 1.00000413e+04, 1.06477420e+04,
        1.13359788e+04, 1.20681156e+04, 1.28474297e+04, 1.36771377e+04,
        1.45604686e+04, 1.55006646e+04, 1.65009805e+04, 1.75646841e+04,
        1.86950558e+04, 1.98954539e+04, 2.11709691e+04],
       [3.90320000e+03, 3.53653741e+03, 3.20656508e+03, 2.91035884e+03,
        2.64520288e+03, 2.40855567e+03, 2.19784496e+03, 2.01073933e+03,
        1.84516909e+03, 1.69921713e+03, 1.57107722e+03, 1.45913918e+03,
        1.36197461e+03, 1.27833357e+03, 1.20704231e+03, 1.14707829e+03,
        1.09754983e+03, 1.05765650e+03, 1.02668919e+03, 1.00403006e+03,
        9.89119335e+02, 9.81445694e+02, 9.80695615e+02, 9.86596493e+02,
        9.98905512e+02, 1.01740965e+03, 1.04192567e+03, 1.07230013e+03,
        1.10840938e+03, 1.15016344e+03, 1.19760749e+03],
       [9.75800000e+02, 1.47397189e+03, 1.89769404e+03, 2.26839811e+03,
        2.60424573e+03, 2.91996279e+03, 3.22336800e+03, 3.52127809e+03,
        3.82056076e+03, 4.12647147e+03, 4.44129848e+03, 4.76749990e+03,
        5.10804586e+03, 5.46638878e+03, 5.84486742e+03, 6.24455402e+03,
        6.66693956e+03, 7.11389127e+03, 7.58765267e+03, 8.09084356e+03,
        8.62618163e+03, 9.19527566e+03, 9.79995276e+03, 1.04423392e+04,
        1.11248193e+04, 1.18500350e+04, 1.26208864e+04, 1.34405315e+04,
        1.43123858e+04, 1.52400863e+04, 1.62265734e+04],
       [6.97000000e+02, 7.14612124e+02, 7.41965200e+02, 7.76856630e+02,
        8.17504296e+02, 8.62575091e+02, 9.11682622e+02, 9.64554029e+02,
        1.02088760e+03, 1.08059494e+03, 1.14398769e+03, 1.21134936e+03,
        1.28289413e+03, 1.35877158e+03, 1.43930077e+03, 1.52497198e+03,
        1.61623523e+03, 1.71351635e+03, 1.81721699e+03, 1.92771462e+03,
        2.04541054e+03, 2.17089424e+03, 2.30472400e+03, 2.44747179e+03,
        2.59973262e+03, 2.76212447e+03, 2.93528829e+03, 3.11988807e+03,
        3.31661075e+03, 3.52617097e+03, 3.74943247e+03],
       [2.30000000e+01, 5.46973565e+02, 1.07315654e+03, 1.60136925e+03,
        2.13193173e+03, 2.66572005e+03, 3.20492153e+03, 3.75181573e+03,
        4.30858085e+03, 4.87766972e+03, 5.46207476e+03, 6.06473871e+03,
        6.68851278e+03, 7.33616520e+03, 8.01075434e+03, 8.71557699e+03,
        9.45393342e+03, 1.02291806e+04, 1.10447320e+04, 1.19040581e+04,
        1.28107893e+04, 1.37689532e+04, 1.47825321e+04, 1.58557228e+04,
        1.69929618e+04, 1.81989258e+04, 1.94785314e+04, 2.08369355e+04,
        2.22795345e+04, 2.38119724e+04, 2.54403514e+04],
       [1.93950800e+05, 1.94132725e+05, 1.94237382e+05, 1.94285950e+05,
        1.94298433e+05, 1.94291373e+05, 1.94266831e+05, 1.94228260e+05,
        1.94181258e+05, 1.94128920e+05, 1.94069758e+05, 1.94003491e+05,
        1.93930991e+05, 1.93854218e+05, 1.93773154e+05, 1.93686201e+05,
        1.93592533e+05, 1.93491746e+05, 1.93383857e+05, 1.93269306e+05,
        1.93148739e+05, 1.93021207e+05, 1.92885089e+05, 1.92739106e+05,
        1.92582324e+05, 1.92414157e+05, 1.92234361e+05, 1.92043043e+05,
        1.91840653e+05, 1.91627844e+05, 1.91401929e+05],
       [2.92549200e+05, 2.92323148e+05, 2.92161782e+05, 2.92045458e+05,
        2.91955389e+05, 2.91875914e+05, 2.91805190e+05, 2.91739863e+05,
        2.91674412e+05, 2.91605636e+05, 2.91534608e+05, 2.91461174e+05,
        2.91384034e+05, 2.91300805e+05, 2.91210931e+05, 2.91115279e+05,
        2.91013930e+05, 2.90906518e+05, 2.90792238e+05, 2.90669837e+05,
        2.90537801e+05, 2.90396034e+05, 2.90245089e+05, 2.90085121e+05,
        2.89915874e+05, 2.89736683e+05, 2.89546473e+05, 2.89343761e+05,
        2.89126650e+05, 2.88892976e+05, 2.88643736e+05],
       [5.04000000e+02, 4.78864711e+02, 4.68710356e+02, 4.70264522e+02,
        4.80801599e+02, 4.98170627e+02, 5.21378625e+02, 5.49611872e+02,
        5.82057911e+02, 6.18187623e+02, 6.57983974e+02, 7.01418411e+02,
        7.48395012e+02, 7.98755710e+02, 8.52553975e+02, 9.10074242e+02,
        9.71556516e+02, 1.03720724e+03, 1.10719931e+03, 1.18167205e+03,
        1.26078418e+03, 1.34491828e+03, 1.43443354e+03, 1.52968378e+03,
        1.63102655e+03, 1.73882309e+03, 1.85343837e+03, 1.97524105e+03,
        2.10460351e+03, 2.24190780e+03, 2.38769625e+03],
       [4.03200000e+02, 3.99352090e+02, 3.95431020e+02, 3.91541824e+02,
        3.87774702e+02, 3.84204189e+02, 3.80872413e+02, 3.77817464e+02,
        3.75078470e+02, 3.72687611e+02, 3.70663616e+02, 3.69026900e+02,
        3.67801225e+02, 3.67013565e+02, 3.66686517e+02, 3.66837541e+02,
        3.67487201e+02, 3.68659011e+02, 3.70379437e+02, 3.72677892e+02,
        3.75585446e+02, 3.79129233e+02, 3.83338903e+02, 3.88247133e+02,
        3.93889429e+02, 4.00304130e+02, 4.07532408e+02, 4.15618267e+02,
        4.24608543e+02, 4.34552778e+02, 4.45500288e+02],
       [1.00800000e+02, 1.52283964e+02, 1.96191362e+02, 2.34800460e+02,
        2.70041601e+02, 3.03478438e+02, 3.35929885e+02, 3.68110446e+02,
        4.00744003e+02, 4.34382706e+02, 4.69260357e+02, 5.05631877e+02,
        5.43809882e+02, 5.84161417e+02, 6.26934169e+02, 6.72242346e+02,
        7.20247272e+02, 7.71151855e+02, 8.25200584e+02, 8.82679530e+02,
        9.43883492e+02, 1.00898778e+03, 1.07820558e+03, 1.15178108e+03,
        1.22998391e+03, 1.31310919e+03, 1.40147750e+03, 1.49543489e+03,
        1.59535286e+03, 1.70162528e+03, 1.81459060e+03],
       [7.20000000e+01, 7.39880706e+01, 7.72771323e+01, 8.15867344e+01,
        8.66876437e+01, 9.24045506e+01, 9.86700692e+01, 1.05433123e+02,
        1.12642815e+02, 1.20274575e+02, 1.28351040e+02, 1.36894014e+02,
        1.45919467e+02, 1.55438024e+02, 1.65480396e+02, 1.76098657e+02,
        1.87341606e+02, 1.99256033e+02, 2.11886720e+02, 2.25276443e+02,
        2.39471351e+02, 2.54539626e+02, 2.70546656e+02, 2.87559643e+02,
        3.05648633e+02, 3.24886508e+02, 3.45348993e+02, 3.67114654e+02,
        3.90264895e+02, 4.14884440e+02, 4.41073801e+02],
       [2.00000000e+00, 2.18917385e+01, 4.55924045e+01, 7.27535442e+01,
        1.03108662e+02, 1.36477173e+02, 1.72840655e+02, 2.12214195e+02,
        2.54623849e+02, 3.00143805e+02, 3.48925892e+02, 4.01132792e+02,
        4.56931310e+02, 5.16493124e+02, 5.80031899e+02, 6.47803988e+02,
        7.20076128e+02, 7.97128171e+02, 8.79253084e+02, 9.66756950e+02,
        1.05996866e+03, 1.15926800e+03, 1.26504386e+03, 1.37771116e+03,
        1.49771301e+03, 1.62552068e+03, 1.76163361e+03, 1.90657943e+03,
        2.06091393e+03, 2.22522208e+03, 2.40014644e+03]])

In [272]:
len(dados1)

14

In [271]:
colors

['blue', 'red', 'green', 'orange', 'grey']

In [291]:
fig = make_subplots(rows=1, cols = 2, column_width = [1,1], shared_xaxes= True, subplot_titles=(
    'Rio de Janeiro', 
    'Niterói'))
for i in range(5):
    fig.add_trace(go.Scatter(
    x=np.arange(len(dados1[i])),
    y=np.abs(dados1[i+2]-dados2[i+2]), name = keys[i+2], marker = dict(color = colors[i], coloraxis = 'coloraxis'), showlegend=False), row = 1, col = 1)
    
for i in range(5):
    fig.add_trace(go.Scatter(
    x=np.arange(len(dados2[i])),
    y=np.abs(dados1[i+9]-dados2[i+9]), name = keys[i+2], marker = dict(color = colors[i])), row = 1, col =2)
    
fig.update_layout(
    shapes=[
        # 1st highlight during Feb 4 - Feb 6
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x1",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=5,
            y0=0,
            x1=7,
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0
        ),
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x2",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=5,
            y0=0,
            x1=7,
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
    ]
)
fig.update_layout(height=500, width=1000, yaxis=dict(type='log'), yaxis2=dict(type='log'), title = 
                  'Comparação entre regimes: abrir por dois dias ou não abrir')
fig.show()